In [1]:
from jimutmap import api

In [12]:
download_obj = api(min_lat_deg = 34.07591649111752,
                   max_lat_deg = 34.07736874425625,
                   min_lon_deg = -84.25748080185113,
                   max_lon_deg = -84.25579308490153,
                   zoom = 23,
                   verbose = False,
                   threads_ = 5, 
                   container_dir = "Dataset")

In [13]:
download_obj.download(getMasks = True)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


In [ ]:
import urllib
import Image
import os
import math

In [14]:
def getXY(self):
    """
        Generates an X,Y tile coordinate based on the latitude, longitude 
        and zoom level
        Returns:    An X,Y tile coordinate
    """
    tile_size = 256

    # Use a left shift to get the power of 2
    # i.e. a zoom level of 2 will have 2^2 = 4 tiles
    numTiles = 1 << self._zoom

    # Find the x_point given the longitude
    point_x = (tile_size/ 2 + self._lng * tile_size / 360.0) * numTiles // tile_size

    # Convert the latitude to radians and take the sine
    sin_y = math.sin(self._lat * (math.pi / 180.0))

    # Calulate the y coorindate
    point_y = ((tile_size / 2) + 0.5 * math.log((1+sin_y)/(1-sin_y)) * -(tile_size / (2 * math.pi))) * numTiles // tile_size

    return int(point_x), int(point_y)

def generateImage(self, **kwargs):
    """
        Generates an image by stitching a number of google map tiles together.

        Args:
            start_x:        The top-left x-tile coordinate
            start_y:        The top-left y-tile coordinate
            tile_width:     The number of tiles wide the image should be -
                            defaults to 5
            tile_height:    The number of tiles high the image should be -
                            defaults to 5
        Returns:
            A high-resolution Goole Map image.
    """

    start_x = kwargs.get('start_x', None)
    start_y = kwargs.get('start_y', None)
    tile_width = kwargs.get('tile_width', 5)
    tile_height = kwargs.get('tile_height', 5)

    # Check that we have x and y tile coordinates
    if start_x == None or start_y == None :
        start_x, start_y = self.getXY()

    # Determine the size of the image
    width, height = 256 * tile_width, 256 * tile_height

    #Create a new image of the size require
    map_img = Image.new('RGB', (width,height))

    for x in range(0, tile_width):
        for y in range(0, tile_height) :
            url = 'https://mt0.google.com/vt?x='+str(start_x+x)+'&y='+str(start_y+y)+'&z='+str(self._zoom)

            current_tile = str(x)+'-'+str(y)
            urllib.urlretrieve(url, current_tile)

            im = Image.open(current_tile)
            map_img.paste(im, (x*256, y*256))

            os.remove(current_tile)

    return map_img

In [16]:
import ee

In [17]:
ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB').filterDate('2018-01-01', '2019-01-01');

EEException: Earth Engine client library not initialized. Run `ee.Initialize()`

In [33]:
# Import earthengine API
import ee
import pprint

In [19]:
# Initialise
ee.Initialize()

# Configure the pretty printing output & initialize earthengine.
pp = pprint.PrettyPrinter(depth=4)

In [40]:
# Load a Landsat 8 ImageCollection for a single path-row.
collection = (ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB').filterDate('2016-03-01', '2018-03-01'))
#pp.pprint('Collection: '+str(collection.getInfo())+'\n')

# Get the number of images.
count = collection.size()
print('Count: ', str(count.getInfo()))

Count:  220


In [26]:
# Get the date range of images in the collection.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', str(ee.Date(range.get('min')).getInfo()), str(ee.Date(range.get('max')).getInfo())+'\n')

# Get statistics for a property of the images in the collection.
sunStats = collection.aggregate_stats('SUN_ELEVATION')
pp.pprint('Sun elevation statistics: '+str(sunStats.getInfo())+'\n')

Date range:  {'type': 'Date', 'value': 1456905747000} {'type': 'Date', 'value': 1483751094000}

("Sun elevation statistics: {'max': None, 'mean': 0, 'min': None, 'sample_sd': "
 "0, 'sample_var': 0, 'sum': 0, 'sum_sq': 0, 'total_count': 0, 'total_sd': 0, "
 "'total_var': 0, 'valid_count': 0, 'weight_sum': 0, 'weighted_sum': 0}\n")


In [53]:
# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort('CLOUD_COVER').first())
print('Least cloudy image: ', str(image.getInfo())+'\n')

# Limit the collection to the 10 most recent images.
recent = collection.sort('system:time_start', False).limit(10)
#pp.pprint('Recent images: '+str(recent.getInfo())+'\n')

Least cloudy image:  {'type': 'Image', 'bands': [{'id': 'R', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [13440, 26880], 'crs': 'PROJCS["UTM Zone 37, Northern Hemisphere", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      TOWGS84[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Transverse_Mercator"], \n  PARAMETER["central_meridian", 39.0], \n  PARAMETER["latitude_of_origin", 0.0], \n  PARAMETER["scale_factor", 0.9996], \n  PARAMETER["false_easting", 500000.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]', 'crs_transform': [0.8, 0, 375909.6, 0, -0.8, 255889.6]}, {'id': '

In [55]:
display(image)

In [28]:
# Get size of collection in bytes
print('Total size of Collection with 10 most recent images : '+str((recent.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))

Total size of Collection with 10 most recent images : 5033610234


In [32]:
geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])
llx = 116.2621
lly = 39.8412
urx = 116.4849
ury = 40.01236
geometry = [[llx,lly], [llx,ury], [urx,ury], [urx,lly]]

task_config = {
    'description': 'imageToDriveExample',
    'scale': 30,  
    'region': geometry
    }

task = ee.batch.Export.image(collection.toBands(), 'exportExample', task_config)

task.start()

TypeError: toDrive() got multiple values for argument 'description'

In [36]:
from geetools import batch
import ee

ee.Initialize()

collection = ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB').filterDate('2018-01-01', '2019-01-01')
tasks = batch.ImageCollection.toDrive(reference, 'MyFolder', scale=5000)

AttributeError: module 'geetools.batch' has no attribute 'ImageCollection'

In [41]:
batch.imagecollection.toDrive(collection, folder='Dataset', verbose=True)

exporting s01_20160302T081654Z to folder 'Dataset' in GDrive
exporting s01_20160302T081735Z to folder 'Dataset' in GDrive
exporting s01_20160302T081910Z to folder 'Dataset' in GDrive
exporting s01_20160302T081954Z to folder 'Dataset' in GDrive
exporting s01_20160303T144544Z to folder 'Dataset' in GDrive
exporting s01_20160304T082423Z to folder 'Dataset' in GDrive
exporting s01_20160306T083026Z to folder 'Dataset' in GDrive
exporting s01_20160307T020720Z to folder 'Dataset' in GDrive
exporting s01_20160307T145511Z to folder 'Dataset' in GDrive
exporting s01_20160307T145551Z to folder 'Dataset' in GDrive
exporting s01_20160308T082955Z to folder 'Dataset' in GDrive
exporting s01_20160308T083333Z to folder 'Dataset' in GDrive
exporting s01_20160309T212126Z to folder 'Dataset' in GDrive
exporting s01_20160310T212911Z to folder 'Dataset' in GDrive
exporting s01_20160314T213956Z to folder 'Dataset' in GDrive
exporting s01_20160326T104737Z to folder 'Dataset' in GDrive
exporting s01_20160328T0

exporting s02_20160803T223319Z to folder 'Dataset' in GDrive
exporting s02_20160804T015703Z to folder 'Dataset' in GDrive
exporting s02_20160808T223332Z to folder 'Dataset' in GDrive
exporting s02_20160817T221451Z to folder 'Dataset' in GDrive
exporting s02_20160819T015811Z to folder 'Dataset' in GDrive
exporting s02_20160823T223405Z to folder 'Dataset' in GDrive
exporting s02_20160827T011401Z to folder 'Dataset' in GDrive
exporting s02_20160902T094203Z to folder 'Dataset' in GDrive
exporting s02_20160902T223413Z to folder 'Dataset' in GDrive
exporting s02_20160903T015851Z to folder 'Dataset' in GDrive
exporting s02_20160903T100125Z to folder 'Dataset' in GDrive
exporting s02_20160907T094218Z to folder 'Dataset' in GDrive
exporting s02_20160908T015841Z to folder 'Dataset' in GDrive
exporting s02_20160908T100132Z to folder 'Dataset' in GDrive
exporting s02_20160911T011357Z to folder 'Dataset' in GDrive
exporting s02_20160912T013301Z to folder 'Dataset' in GDrive
exporting s02_20160924T0

[<Task F4I6I2AB4RX5PIORFG6O3HKZ EXPORT_IMAGE: s01_20160302T081654Z (UNSUBMITTED)>,
 <Task R6HYD4P3WPJBO4J5EPMQJZOR EXPORT_IMAGE: s01_20160302T081735Z (UNSUBMITTED)>,
 <Task IGKJWBUHFRGTSLRVRPSCOBR3 EXPORT_IMAGE: s01_20160302T081910Z (UNSUBMITTED)>,
 <Task EYQMVV5ONBA5FPVCQZKSRHAL EXPORT_IMAGE: s01_20160302T081954Z (UNSUBMITTED)>,
 <Task EJIWI6POZ36I6FPNOFHXOBBZ EXPORT_IMAGE: s01_20160303T144544Z (UNSUBMITTED)>,
 <Task 7K7IARGTD3ST7ELMKDYKCIKN EXPORT_IMAGE: s01_20160304T082423Z (UNSUBMITTED)>,
 <Task COYUJBJWRBG2ATMRGGKGS5NF EXPORT_IMAGE: s01_20160306T083026Z (UNSUBMITTED)>,
 <Task MDQIVD7Y6BW6MUOXKGTLY25O EXPORT_IMAGE: s01_20160307T020720Z (UNSUBMITTED)>,
 <Task HGVY4XF6ALLWX7WNGX3XP42Y EXPORT_IMAGE: s01_20160307T145511Z (UNSUBMITTED)>,
 <Task T4HNL2XW6MP2D6M4AYLFLSYH EXPORT_IMAGE: s01_20160307T145551Z (UNSUBMITTED)>,
 <Task 6CCCGDQXNROBCXP67A4F43KX EXPORT_IMAGE: s01_20160308T082955Z (UNSUBMITTED)>,
 <Task RUBIXH4FVP4D3IYKMZOKQ4BQ EXPORT_IMAGE: s01_20160308T083333Z (UNSUBMITTED)>,
 <Ta

In [43]:
import folium

In [44]:
first = (ee.ImageCollection('COPERNICUS/S2_SR')
         .filterBounds(ee.Geometry.Point(-70.48, 43.3631))
         .filterDate('2019-01-01', '2019-12-31')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first())

# Define a map centered on southern Maine.
map_s2 = folium.Map(location=[43.7516, -70.8155], zoom_start=11)

# Add the image layer to the map and display it.
map_s2.add_ee_layer(
    first, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'first')
display(map_s2)

AttributeError: 'Map' object has no attribute 'add_ee_layer'

In [45]:
first = (ee.ImageCollection('COPERNICUS/S2_SR')
         .filterBounds(ee.Geometry.Point(-70.48, 43.3631))
         .filterDate('2019-01-01', '2019-12-31')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first())

In [51]:
image

In [52]:
import ee
from IPython.display import Image,display
ee.Initialize()
image = ee.Image('srtm90_v4')
url = image.getThumbURL({'min':0,'max':3000})
Image(url=url) 

EEException: Pixel grid dimensions (432000x144000) must be less than or equal to 10000.